# Summary

## importaciones
nota: estas importaciones son necesarias para este notebook, si quiere ver las importaciones necesarias para la función especifica vaya al archivo original [summary](../features/summary.py)

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
spark = SparkSession.builder.getOrCreate()

## Dataframes y validaciones 
Estos son dataframes de testeo obtenidos de internet. 
Las validaciones son una pieza importante de la parte tecnica de la libreria. Para mas detalle vea las [validaciones](../validaciones/validaciones.py)

In [2]:
df_cars = spark.read.csv('csv/used_cars_data.csv' , header=True,inferSchema=True )
df_countries = spark.read.csv('csv/countries.csv' ,header= True ,inferSchema=True )
df_dates = spark.read.csv('csv/US_Holiday_Dates_(2004-2021).csv' , header=True,inferSchema=True )
df_cop = spark.read.csv('csv/eurocup_2020_results.csv', header= True,inferSchema=True)
df_test = spark.read.csv('csv/Countries_usefulFeatures.csv' , header= True ,inferSchema=True)
df_ernigs = spark.read.csv('csv/all_earnings_dates.csv' , header= True ,inferSchema=True)
df_null = spark.read.csv('csv/cars_null.csv' , header= True  ,inferSchema=True)
df_seph = spark.read.csv('csv/sephora_website_dataset.csv', header= True,inferSchema=True)
df_tw = spark.read.csv('csv/most_followed_twitter.csv' , header= True ,inferSchema=True)
df_airlines = spark.read.csv('csv/airlines.csv' , header= True ,inferSchema=True)

def is_dataframe(dataframe):
    try:
        type_df = type(dataframe)
        if not isinstance(dataframe , DataFrame):
            raise TypeError(f"Expected a DataFrame, got {type_df}")
    except Exception as e: 
        print("An error occurred: ", e)
def sel_num_cols(dataframe): 
    lista_columnas_numericas =  []
    tipos_numericos = [LongType().simpleString(), DoubleType().simpleString(), 
        IntegerType().simpleString() , ShortType().simpleString() ,
        FloatType().simpleString() , DecimalType().simpleString()]
    for columnas, dtype in dataframe.dtypes:
        if dtype in tipos_numericos: lista_columnas_numericas.append(columnas)
    
    return lista_columnas_numericas
def df_has_numtype(dataframe):
    try:
        is_dataframe(dataframe)
        tipos_numericos = [LongType().simpleString(), DoubleType().simpleString(), 
        IntegerType().simpleString() , ShortType().simpleString() ,
        FloatType().simpleString() , DecimalType().simpleString()]
        for _ , dtype in dataframe.dtypes: 
            if dtype in tipos_numericos:  return True 
        return False
    except Exception as e:
        print("An error occurred: ", e)
def df_has_null(dataframe): 
    try:    
        is_dataframe(dataframe)
        for col in dataframe.columns:
            if dataframe.filter(isnull(col)).count() > 0 : return (True , col)
        return (False , None)
    except Exception as e: 
        print("An error occurred: ", e)

## Summary

In [3]:
def sumary(dataframe ):
    """
    Esta función recibe un dataframe de PySpark. 
    La función devuelve un dataframe con un resumen estadistico de todas las
    columnas de tipo numericas, este resumen toma como base a la funcíon Summary()
    de pyspark y le agrega la moda y antimoda al dataframe. En caso de que no haya 
    columnas tipo numericas la función no hara nada y retornara el mismo dataframe entregado.
    
    Argumento:
    dataframe (pyspark.sql.dataframe.DataFrame): Dataframe que desea obtener datos estadisticos.
    
    Retorno:
    dataframe (pyspark.sql.dataframe.DataFrame): Dataframe de PySpark con el resumen estadistico mejorado en caso de tener columnas de tipo numericas.
    """
    is_dataframe(dataframe) 
    if df_has_numtype(dataframe):
        num_cols = sel_num_cols(dataframe)
        df_columns = dataframe.select(*num_cols)
        lista_modas= ['moda']
        lista_antimoda = ['antimoda']
        lista_columnas = ['summary']
        resultado , _  = df_has_null(dataframe)
        mensaje = 'las columnas numericas tienen datos nulos' if resultado else 'columnas sin datos nulos'
        print(mensaje)
        for col in num_cols:
            mode_val = dataframe.groupBy(col).count().sort('count', ascending=False).first()[0]
            anti_mode_val = dataframe.groupBy(col).count().sort('count', ascending=True).first()[0]
            lista_modas.append(mode_val)
            lista_antimoda.append(anti_mode_val)
        for valor in num_cols: lista_columnas.append(valor)
        
        lista_modas = tuple(lista_modas)
        lista_antimoda = tuple(lista_antimoda)
        new_row = spark.createDataFrame([lista_modas] , [*lista_columnas])
        new_row_anti_mode = spark.createDataFrame([lista_antimoda] , [*lista_columnas])
        df_columns.summary().unionAll(new_row).unionAll(new_row_anti_mode).show()
        return df_columns.asummary().unionAll(new_row).unionAll(new_row_anti_mode)
    else : 
        print('El dataframe pyspark propocionado no tiene numeros')
        return dataframe

## Ejemplo en databricks
Este es un ejemplo de esta función ejecutada en databricks
### dataframe instanciado
![dataframe instanciado](../img/dataframe.png)
### dataframe obtenido al aplicar la función
![dataframe obtenido después de haber utilizado la función](../img/summary.png)

## Mas ejemplos:

In [ ]:
df_sumary1 = sumary(df_cars)

In [ ]:
df_sumary2 = sumary(df_airlines)

In [ ]:
df_sumary3 = sumary(df_cop)


In [ ]:
df_sumary4 = sumary(df_countries)

In [ ]:
df_sumary5 = sumary(df_dates)